In [113]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [68]:
pd.set_option('display.max_rows', 200)

0. Data processing  
    - histogram에 대한 처리: 대표값으로 대체
        - 평균, 분산
    - NA 값에 대한 처리: columnwise 적용. pos null 30%이상 -> pos median, 나머지 -> 전체의 median 
    - class imbalance: oversampling (SMOTE)
    - feature selection: drop over 40% na, 2장 내용 참고, logistic 회귀
    - feature reduction: PCA
    - columnwise normalizing

- dtype processing

In [119]:
data = pd.read_csv("./Train_data.csv", index_col=0)

# class to numeric; pos: 1, neg: 0
data['class'] = data['class'].replace('neg', 0)
data['class'] = data['class'].replace('pos', 1)

# na, numeric processing
data = data.replace('na', np.nan) # change 'na' ro np.nan
data = data.apply(pd.to_numeric) # change object to int or float

- na: process columnwise. 
    - Drop if null ratio over 0,4; gap btw 25~40%
    - If pos_null_ratio>=0.3, pos median. Else total median; If pos_null_ratio<0.3, many cols also have high neg_null_ratio
    

In [124]:
# check null columns per cats
pos_data = data[data["class"]==1] 
neg_data = data[data["class"]==0] 

pos_df, neg_df = pos_data.isnull().sum().to_frame(name="pos_null_cnt"), neg_data.isnull().sum().to_frame(name="neg_null_cnt")
null_check = pos_df.join(neg_df)
null_check["total_null"] = null_check["pos_null_cnt"] + null_check["neg_null_cnt"]
null_check.insert(1, "pos_cnt", len(pos_data))
null_check.insert(3, "neg_cnt", len(neg_data))

null_check["pos_null_ratio"] = null_check.pos_null_cnt/null_check.pos_cnt
null_check["neg_null_ratio"] = null_check.neg_null_cnt/null_check.neg_cnt
null_check["total_null_ratio"] = (null_check.pos_null_cnt + null_check.neg_null_cnt)/len(data)

# drop cols w/ too many na's
to_drop = null_check[null_check.total_null_ratio>=0.4]
data.drop(to_drop.index, inplace=True, axis=1)

# na replacement
POS_MEDIAN = pos_data.median(axis=0).to_dict()
TOTAL_MEDIAN = data.median(axis=0).to_dict()

with_pos = data.loc[:, null_check.pos_null_ratio>=0.4]
with_pos = with_pos.index
data.fillna(value=POS_MEDIAN, inplace=True)

with_total = data.loc[:, null_check.pos_null_ratio<0.4]
with_total = with_total.index
data.fillna(value=TOTAL_MEDIAN, inplace=True)

# data.isnull().sum().sum()

- Rowise Standardize

In [128]:
X = data.iloc[:, 1:]
y = data["class"]

scaler = StandardScaler()
scaler.fit_transform(X)
X_scaled = scaler.fit_transform(X) # standardizing if needed
X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
X_scaled.describe()

,aa_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,ag_004,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
count,5.700000e+04,5.700000e+04,5.700000e+04,5.700000e+04,5.700000e+04,5.700000e+04,5.700000e+04,5.700000e+04,5.700000e+04,5.700000e+04,...,5.700000e+04,5.700000e+04,5.700000e+04,5.700000e+04,5.700000e+04,5.700000e+04,5.700000e+04,5.700000e+04,5.700000e+04,5.700000e+04
mean,4.352181e-17,1.023197e-16,-9.134969e-16,1.819474e-15,-2.873165e-17,1.769193e-15,-5.037638e-15,3.297087e-15,-6.553978e-15,3.826715e-17,...,-3.820330e-17,-1.233784e-16,-2.477112e-17,-3.588260e-18,-2.711257e-16,-1.003334e-16,3.149274e-16,1.371934e-15,-1.276193e-15,1.573291e-17
std,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,...,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00
min,-2.628782e-01,-4.329847e-01,-4.202499e-03,-4.091289e-02,-5.254166e-02,-1.142451e-02,-2.377023e-02,-5.658439e-02,-1.182197e-01,-1.924195e-01,...,-3.924092e-01,-3.971752e-01,-3.861963e-01,-3.649720e-01,-3.272284e-01,-2.059314e-01,-2.965220e-01,-1.780667e-01,-1.951579e-02,-2.304906e-02
25%,-2.591822e-01,-4.329847e-01,-4.201331e-03,-4.091289e-02,-5.254166e-02,-1.142451e-02,-2.377023e-02,-5.658439e-02,-1.182197e-01,-1.922981e-01,...,-3.899072e-01,-3.950425e-01,-3.838522e-01,-3.616823e-01,-3.267009e-01,-2.058613e-01,-2.965220e-01,-1.780667e-01,-1.951579e-02,-2.304906e-02
50%,-1.306353e-01,-4.329845e-01,-4.194489e-03,-4.091289e-02,-5.254166e-02,-1.142451e-02,-2.377023e-02,-5.658439e-02,-1.182197e-01,-1.909014e-01,...,-1.972482e-01,-1.970052e-01,-2.020596e-01,-1.994583e-01,-2.356962e-01,-1.806732e-01,-2.869486e-01,-1.780667e-01,-1.951579e-02,-2.304906e-02
75%,-5.297553e-02,-4.329836e-01,-4.184532e-03,-4.091289e-02,-5.254166e-02,-1.142451e-02,-2.377023e-02,-5.658439e-02,-1.182197e-01,-1.712852e-01,...,-2.794715e-02,-8.012186e-03,-9.598749e-04,-1.762090e-02,-6.975556e-02,-1.058132e-01,-6.852790e-03,-1.375054e-01,-1.951579e-02,-2.304906e-02
max,1.838947e+02,2.309554e+00,2.387446e+02,1.277613e+02,9.700465e+01,1.606072e+02,1.817643e+02,1.023440e+02,8.306757e+01,8.771011e+01,...,6.254742e+01,6.507378e+01,7.739346e+01,4.768775e+01,3.775423e+01,6.852664e+01,3.628087e+01,9.821978e+01,1.267962e+02,1.543036e+02


- oversampling; SMOTE